## Exploring CUDA in Colab

Main reference: [[1](#ref-1)]. In this Colab notebook, we will explore CUDA examples.

Google provides free access to an Nvidia Tesla T4 GPU on Colab that can be chosen by navigating to Runtime->Change runtime type.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# from google.colab import drive
# drive.flush_and_unmount()

Mounted at /content/drive


Nvidia's cuda compiler driver `nvcc`  comes pre-installed in Colab's file system, as can be checked by typing `!nvcc --version`

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


Subsequently, we install and load the `nvcc4jupyter` extension as follows:

In [5]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-689dld_q
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-689dld_q
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 801584cceb559adc54e828ebe9b385c5f53fe70f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fm3_pvlk/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpzsp8jg5n".


We are now ready to test CUDA using the `%%cuda` cell magic

In [6]:
%%cuda
#include <iostream>
int main()
{
	std::cout << "Congratulations, CUDA is up and running!\n";
	return 0;
}


Congratulations, CUDA is up and running!



For context, a single Tesla T4 GPU has 40 SMs with 64 cuda cores each (hence 2560 cuda cores in total). With warp-scheduling and latency hiding, a single SM can support a maximum of 1024 concurrent threads (32 warps each with 32 threads). Hopefully we can test the full power of T4 by the end of this notebook!

Let us test a CUDA program that squares every number in the array [0,1, ... 64]

In [7]:
%%cuda
#include <stdio.h>

//a simple kernel that squares every element in an array
__global__ void square(float * d_in, float * d_out) {
    int idx = threadIdx.x;
    float f = d_in[idx];
    d_out[idx] = f * f;
}

//cpu program flow
int main(int argc, char ** argv) {
    const int ARRAY_SIZE = 64;
    const int ARRAY_BYTES = ARRAY_SIZE * sizeof(float);

    //generate input array, declare output array on cpu (host)
    float h_in[ARRAY_SIZE];
    for (int i = 0; i < ARRAY_SIZE; i++) {
        h_in[i] = float(i);
    }
    float h_out[ARRAY_SIZE];

    //declare gpu memory pointers, allocate gpu memory (device)
    float * d_in;
    float * d_out;
    cudaMalloc((void **) &d_in, ARRAY_BYTES);
    cudaMalloc((void **) &d_out, ARRAY_BYTES);

    //transfer input from host to device
    cudaMemcpy(d_in, h_in, ARRAY_BYTES, cudaMemcpyHostToDevice);
    //launch kernel on device
    square<<<1, ARRAY_SIZE>>>(d_in, d_out);
    //transfer output from device to host
    cudaMemcpy(h_out, d_out, ARRAY_BYTES, cudaMemcpyDeviceToHost);

    //print output
    for (int i = 0; i < ARRAY_SIZE; i++) {
        printf("%f", h_out[i]);
        printf(((i%4) != 3) ? "\t" : "\n");
    }

    //free gpu memory
    cudaFree(d_in);
    cudaFree(d_out);

    return 0;
}

0.000000	1.000000	4.000000	9.000000
16.000000	25.000000	36.000000	49.000000
64.000000	81.000000	100.000000	121.000000
144.000000	169.000000	196.000000	225.000000
256.000000	289.000000	324.000000	361.000000
400.000000	441.000000	484.000000	529.000000
576.000000	625.000000	676.000000	729.000000
784.000000	841.000000	900.000000	961.000000
1024.000000	1089.000000	1156.000000	1225.000000
1296.000000	1369.000000	1444.000000	1521.000000
1600.000000	1681.000000	1764.000000	1849.000000
1936.000000	2025.000000	2116.000000	2209.000000
2304.000000	2401.000000	2500.000000	2601.000000
2704.000000	2809.000000	2916.000000	3025.000000
3136.000000	3249.000000	3364.000000	3481.000000
3600.000000	3721.000000	3844.000000	3969.000000



Okay, so CUDA indeed works as we expect it to with respect to the standard program flow! (in sequence: declare CPU i/o memory, initialize inputs in CPU, allocate GPU i/o memory, copy inputs to GPU, compute in GPU, copy outputs back to CPU). We are now prepared to take on more complex tasks!

In [3]:
!pip install colab-xterm
%load_ext colabxterm
%env TERM=xterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 1.9 MB/s eta 0:00:00
env: TERM=xterm


In [ ]:
%xterm

 ## CS344 Intro to Parallel Programming (Problem Sets)

The problem sets of this course are image processing assignments using CUDA and OpenCV. Running these assignments in colab needs using `colab-xterm` whose installation is described in the previous snippets. Once installed, we navigate into a copy of [this fork of CS344](https://github.com/Adeemj/cs344) and follow instructions in the README to install and build OpenCV in the repo folder.

After the above is done, for each problem we solve (by writing a kernel in `student_func.cu`), we run `make` and `./HWk input.jpg` where "k" indexes Problem Set k whose folder we are in). Here we present a summary of our results.

We successfully solved and ran Problem 1. Problem 1 asks us to write a simple kernel that converts color images to greyscale images by weighted averaging the RGB channels in each pixel as output = 0.299f * R + 0.587f * G + 0.114f * B. We launch the kernel below as a parallel computation of `numRows` blocks each with `numCols` threads in one dimension.

```
__global__
void rgba_to_greyscale(const uchar4* const rgbaImage, unsigned char* const greyImage, int numRows, int numCols)
{
  int idx = threadIdx.x + (blockIdx.x * numCols);
  float val = (.299f * rgbaImage[idx].x) + (.587f * rgbaImage[idx].y) + (.114f * rgbaImage[idx].z);
  greyImage[idx] = val;
}

void your_rgba_to_greyscale(const uchar4 * const h_rgbaImage, uchar4 * const d_rgbaImage, unsigned char* const d_greyImage, size_t numRows, size_t numCols)
{
  const dim3 blockSize(numCols, 1, 1);  
  const dim3 gridSize(numRows, 1, 1);  
  rgba_to_greyscale<<<gridSize, blockSize>>>(d_rgbaImage, d_greyImage, numRows, numCols);
  cudaDeviceSynchronize(); checkCudaErrors(cudaGetLastError());
}
```

Input:

<img src="https://drive.google.com/uc?export=view&id=1lLxx9JxLuMZqrKufokpC17HTWRhJFH5I" width="500px">

Output:

<img src="https://drive.google.com/uc?export=view&id=1X_VABMZxBYLyrOmhPGOIuRVFMOZ9xifx" width="500px">

Reference:

<img src="https://drive.google.com/uc?export=view&id=1fiFTfDfZwF11ctiQdEtSWYHWfZ9BUXPy" width="500px">

Difference (between reference and output):

<img src="https://drive.google.com/uc?export=view&id=1ZdOVvEiBo79Yh9KTtnxGCnnGp_CxybhZ" width="500px">


We notice a small number of isolated white pixels in the difference (like stars in the night sky!). It is unclear what the basis of this miniscule error is. Perhaps they were introduced intentionally, but not sure!


In Problem 2, we are asked to implement [Gaussian Blurring](https://en.wikipedia.org/wiki/Gaussian_blur) of the same image using a 9x9 filter. This is a slightly more involved task. We omit the full code here for brevity, and focus on the results.

xterm window:

<img src="https://drive.google.com/uc?export=view&id=1NVQErePZuTnsZtBvU_GIucgIxt7b4IY0" width="500px">

Input:

<img src="https://drive.google.com/uc?export=view&id=1lLxx9JxLuMZqrKufokpC17HTWRhJFH5I" width="500px">

Output:

<img src="https://drive.google.com/uc?export=view&id=13Vy9seUNQU5Gzv1YDWIYm_s1a6_1bBop" width="500px">

Reference:

<img src="https://drive.google.com/uc?export=view&id=13SSI5x_-CnaVXyF988lskSjDRhgkEKIM" width="500px">

Difference (between reference and output):

<img src="https://drive.google.com/uc?export=view&id=13OlDGjoC3ECy9T61kmcN_-XULU9A5if1" width="500px">


We got it perfectly correct this time. Basically a 9x9 (9 is the default filter width they choose) matrix of weights is centered at each pixel and averaged over its neighborhood. The main `gaussian_blur` kernel was written as follows:
```
__global__
void gaussian_blur(const unsigned char* const inputChannel,
                   unsigned char* const outputChannel,
                   int numRows, int numCols,
                   const float* const filter, const int filterWidth)
{
  // Calculate thread position
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  
  if (col < numCols && row < numRows)
  {
    float result = 0.0f;
    // For each value in the filter
    for (int i = 0; i < filterWidth; i++)
    {
      for (int j = 0; j < filterWidth; j++)
      {
        // Find the global image position for this filter position
        // Clamp to boundary of the image
        int iRow = min(max(row + i - filterWidth/2, 0), numRows-1);
        int iCol = min(max(col + j - filterWidth/2, 0), numCols-1);
        
        result += inputChannel[iRow*numCols + iCol] * filter[i*filterWidth + j];
      }
    }
    // Write the new value to the output
    outputChannel[row*numCols + col] = result;
  }
  ```

---
### References  
[1] <a id="#ref-1"></a> [https://github.com/Adeemj/cs344](https://github.com/Adeemj/cs344)  